In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
df_train = pd.read_csv('mobile_price_classif/train.csv')

In [3]:
df_train.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [4]:
df_train

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [5]:
X = df_train.drop(labels=['price_range'], axis=1)
X = X.to_numpy(dtype=np.int32)
X, X.shape

(array([[ 842,    0,    2, ...,    0,    0,    1],
        [1021,    1,    0, ...,    1,    1,    0],
        [ 563,    1,    0, ...,    1,    1,    0],
        ...,
        [1911,    0,    0, ...,    1,    1,    0],
        [1512,    0,    0, ...,    1,    1,    1],
        [ 510,    1,    2, ...,    1,    1,    1]], dtype=int32),
 (2000, 20))

In [6]:
y = df_train['price_range']
y = y.to_numpy()
y

array([1, 2, 2, ..., 3, 0, 3])

In [7]:
y_classes = df_train.groupby('price_range').ngroups
y_classes

4

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [9]:
## data to torch 

X_train = torch.from_numpy(X_train).type(torch.float).to(device)
y_train = torch.from_numpy(y_train).type(torch.long).to(device)
X_test = torch.from_numpy(X_test).type(torch.float).to(device)
y_test = torch.from_numpy(y_test).type(torch.long).to(device)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([1600, 20]),
 torch.Size([1600]),
 torch.Size([400, 20]),
 torch.Size([400]))

In [10]:
# model

class MobileClassif(nn.Module):
    def __init__(self, inp_shape, out_shape):
        super().__init__()
        
        self.l1 = nn.Linear(in_features=inp_shape, out_features=50)
        self.l2 = nn.Linear(in_features=50, out_features=100)          
        self.l3 = nn.Linear(in_features=100, out_features=50)          
        self.l4 = nn.Linear(in_features=50, out_features=out_shape)
        self.relu = nn.ReLU()
        self.lk_relu = nn.LeakyReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.l1(x)
        x = self.lk_relu(x)
        x = self.l2(x)  
        x = self.lk_relu(x)
        x = self.l3(x)
        x = self.lk_relu(x)
        x = self.l4(x)
        #x = self.relu(x)
        
#         x = self.tanh(x)
        return x

In [11]:
model_0 = MobileClassif(inp_shape=X_train.shape[1], out_shape=y_classes)

In [12]:
model_0.to(device)
next(model_0.parameters()).device

device(type='cuda', index=0)

In [13]:
loss_fn = nn.CrossEntropyLoss()
optim_fn = torch.optim.Adam(params=model_0.parameters(), lr=0.01)

In [14]:
loss_fn, optim_fn

(CrossEntropyLoss(),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.01
     maximize: False
     weight_decay: 0
 ))

In [15]:
model_0(X_train)

tensor([[-22.7936,  16.0975,  34.8216,  29.0932],
        [-50.6433,   8.2120,  57.5859,  14.7799],
        [-18.3074,   8.1299,  28.2424,  15.0176],
        ...,
        [-27.7838,  12.6208,  44.2479,  22.6335],
        [-41.2690,   9.8125,  55.3604,  22.5937],
        [-30.2728,   3.2233,  34.1966,  11.7947]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [16]:
## train loop

epoch = 2000

for e in range(epoch):
    model_0.train()
    y_logits = model_0(X_train).squeeze()
    loss = loss_fn(y_logits, y_train)
#     print("Form oloass", loss.cpu().detach().numpy())
    optim_fn.zero_grad()
    loss.backward()
    optim_fn.step()
    model_0.eval()
    
    if e % 100 == 0:
        with torch.inference_mode():
            print(f"Epoch: {e}, Loss: {loss.cpu().detach().numpy()}")

Epoch: 0, Loss: 26.891128540039062
Epoch: 100, Loss: 0.7164996266365051
Epoch: 200, Loss: 0.4952300786972046
Epoch: 300, Loss: 0.412321537733078
Epoch: 400, Loss: 0.4583348333835602
Epoch: 500, Loss: 0.35342442989349365
Epoch: 600, Loss: 0.3077812194824219
Epoch: 700, Loss: 0.5890491008758545
Epoch: 800, Loss: 0.3426574766635895
Epoch: 900, Loss: 0.296581506729126
Epoch: 1000, Loss: 0.334269255399704
Epoch: 1100, Loss: 0.25399771332740784
Epoch: 1200, Loss: 0.37553155422210693
Epoch: 1300, Loss: 0.3275289237499237
Epoch: 1400, Loss: 0.27626562118530273
Epoch: 1500, Loss: 0.27853378653526306
Epoch: 1600, Loss: 0.4505046010017395
Epoch: 1700, Loss: 0.20577998459339142
Epoch: 1800, Loss: 0.18320207297801971
Epoch: 1900, Loss: 0.17738939821720123


In [17]:
y_preds_test = model_0(X_test).cpu().detach().numpy()
y_preds_test

array([[ 28.950047 ,  14.027866 , -15.09871  , -32.655876 ],
       [ -8.792676 ,   6.827611 ,   8.10074  ,   4.3978605],
       [ 17.034319 ,  19.437035 ,  -1.3237883, -32.433105 ],
       ...,
       [  5.482933 ,   7.573552 ,  -9.708184 ,  -6.543363 ],
       [ -1.4211953,   7.9572706,   7.185425 ,  -6.7615347],
       [ -0.8450434,   6.021343 ,   7.9484763,  -5.945356 ]],
      dtype=float32)

In [18]:
y_pds_test = []

for i in y_preds_test:
    y_pds_test.append(np.argmax(i))

# y_pds

In [19]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test.cpu().detach().numpy(), y_pds_test)

0.805

In [20]:
y_preds_train = model_0(X_train).cpu().detach().numpy()

y_pds_train = []

for i in y_preds_train:
    y_pds_train.append(np.argmax(i))

accuracy_score(y_train.cpu().detach().numpy(), y_pds_train)

0.94625